In [1]:
from aocd import get_data

puzzle_input = get_data(day=10, year=2023)

In [2]:
it1 = """
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
""".strip()

In [24]:
DIRS: dict[str, tuple[int, int]] = {
    "W": (0, -1),
    "E": (0, 1),
    "N": (-1, 0),
    "S": (1, 0),
}
PIPES: dict[str, set[str]] = {
    "-": {"W", "E"},
    "|": {"N", "S"},
    "7": {"W", "S"},
    "J": {"W", "N"},
    "L": {"E", "N"},
    "F": {"E", "S"},
    ".": set(),
}
INV_DIRS: dict[str, str] = {
    "N": "S",
    "E": "W",
    "S": "N",
    "W": "E"
}


def parse_input(input: str):
    rows: list[str] = input.split("\n")

    start = None

    for i, row in enumerate(rows):
        try:
            start = (i, row.index("S"))
            break
        except ValueError:
            pass

    if start is None:
        raise ValueError("Le départ n'a pas été trouvé")

    return start, rows


it1_start, it1_maps = parse_input(it1)
it1_start, it1_maps 

((2, 0), ['7-F7-', '.FJ|7', 'SJLL7', '|F--J', 'LJ.LJ'])

Partie 1

In [25]:
def get_next_dir(
    maps: list[str], pos: tuple[int, int], dir_prec: str | None = None
) -> tuple[tuple[int, int], str]:
    i, j = pos
    if maps[i][j] == "S":
        for di, dj, df in ((-1, 0, "S"), (1, 0, "N"), (0, 1, "W"), (0, -1, "E")):
            ni = i + di
            nj = j + dj
            if (
                (0 <= ni < len(maps))
                and (0 <= nj < len(maps[ni]))
                and df in PIPES[maps[ni][nj]]
            ):
                return (ni, nj), df
        raise Exception(f"{maps}, {pos}, {dir_prec}")
    else:
        assert dir_prec is not None
        (new_dir,) = PIPES[maps[i][j]].difference(dir_prec)
        di, dj = DIRS[new_dir]
        return (i + di, j + dj), INV_DIRS[new_dir]


get_next_dir(it1_maps, (4, 1), "W")

((3, 1), 'S')

In [27]:
def find_main_loop(maps: list[str], start: tuple[int, int]):
    loop = [start]
    dir_prec = None
    current_pos = start
    while True:
        current_pos, dir_prec = get_next_dir(maps, current_pos, dir_prec)
        if current_pos == start:
            break
        else:
            loop.append(current_pos)
    return loop


len(find_main_loop(it1_maps, it1_start))

16

In [29]:
puzzle_start, puzzle_maps = parse_input(puzzle_input)
puzzle_start

(52, 100)

In [31]:
len(find_main_loop(puzzle_maps, puzzle_start)) // 2

6823

Partie 2

In [74]:
def find_main_loop_oriented(maps: list[str], start: tuple[int, int]):
    loop = [start]
    sens = []
    dir_prec = None
    current_pos = start
    while True:
        current_pos, dir_prec = get_next_dir(maps, current_pos, dir_prec)
        if current_pos == start:
            sens.append(INV_DIRS[dir_prec])
            break
        else:
            loop.append(current_pos)
            sens.append(INV_DIRS[dir_prec])
        
    
    return {pos: sens for pos, sens in zip(loop, sens)}


In [116]:
find_main_loop_oriented(it1_maps, it1_start)

{(2, 0): 'S',
 (3, 0): 'S',
 (4, 0): 'E',
 (4, 1): 'N',
 (3, 1): 'E',
 (3, 2): 'E',
 (3, 3): 'E',
 (3, 4): 'N',
 (2, 4): 'W',
 (2, 3): 'N',
 (1, 3): 'N',
 (0, 3): 'W',
 (0, 2): 'S',
 (1, 2): 'W',
 (1, 1): 'S',
 (2, 1): 'W'}

In [62]:
it2 = """
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
""".strip()

it2_start, it2_maps = parse_input(it2)

In [133]:
directions_to_pipe = {
    "NN": "|",
    "NW": "7",
    "NE": "F",

    "SS": "|",
    "SW": "J",
    "SE": "L",

    "EE": "-",
    "EN": "J",
    "ES": "7",

    "WW": "-",
    "WN": "L",
    "WS": "F",
}
def get_start_pipe(loop):
    sens = list(loop.values())
    dir_prev = sens[-1]
    dir = sens[0]
    return directions_to_pipe[f"{dir_prev}{dir}"]

In [134]:

pipe_to_direction = {
    "|": {"N", "S"},
    "-": {"W", "E"},
    "L": {"N", "E"},
    "J": {"N", "W"},
    "7": {"S", "W"},
    "F": {"S", "E"},
}

def get_inner_tiles(start, maps):
    nb_inside = 0

    main_loop = find_main_loop_oriented(maps, start)
    start_pipe = get_start_pipe(main_loop)

    for i, row in enumerate(maps):
        count = 0
        for j, pipe in enumerate(row):
            if (i, j) in main_loop:
                if pipe == "S":
                    pipe = start_pipe
                if "N" in pipe_to_direction[pipe]:
                    count += 1

            elif count % 2 != 0:
                nb_inside += 1

    return nb_inside

get_inner_tiles(it2_start, it2_maps)

8

In [135]:
get_inner_tiles(puzzle_start, puzzle_maps)

415

## Annexes

Approche par agglomération des positions qui ne sont pas dans la loop (ne fonctionne pas car les positions entourés de 
parois de la loop peuvent être à l'extérieur de celle-ci)

In [127]:
def get_neighbors(pos:tuple[int, int]) -> list[tuple[int, int]]:
    i, j = pos
    return [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

def get_inner_tiles(start, maps):
    """NE FONCTIONNE PAS"""
    N = len(maps)
    M = len(maps[0])

    main_loop = find_main_loop_oriented(maps, start)
    all_pos = {(i, j) for i in range(N) for j in range(M)}

    inner = []
    outer = []

    outside_main_loop = all_pos.difference(main_loop)

    while outside_main_loop:
        current_pos = outside_main_loop.pop()

        outside = False
        current_tile = {current_pos}
        current_tile_to_handle = [current_pos]

        while current_tile_to_handle:
            current_pos = current_tile_to_handle.pop()

            for neighbor in get_neighbors(current_pos):
                if neighbor in current_tile:
                    continue
                elif neighbor in main_loop:
                    continue
                elif neighbor not in all_pos:
                    outside = True
                else:
                    current_tile.add(neighbor)
                    current_tile_to_handle.append(neighbor)

        if outside:
            outer.append(current_tile)
        else:
            inner.append(current_tile)

        outside_main_loop.difference_update(current_tile)

    return inner

In [ ]:
sum(map(len, get_inner_tiles(it2_start, it2_maps)))